In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from dautil import report
from dautil import plotting
import numpy as np
from tabulate import tabulate
import urllib

In [ ]:
def aggregate():
    df = sns.load_dataset("anscombe")

    agg = df.groupby('dataset')\
             .agg([np.mean, np.var])\
             .transpose()
    groups = df.groupby('dataset')

    corr = [g.corr()['x'][1] for _, g in groups]
    builder = report.DFBuilder(agg.columns)
    builder.row(corr)

    fits = [np.polyfit(g['x'], g['y'], 1) for _, g in groups]
    builder.row([f[0] for f in fits])
    builder.row([f[1] for f in fits])
    bottom = builder.build(['corr', 'slope', 'intercept'])

    return df, pd.concat((agg, bottom))

In [ ]:
def generate(table):
    writer = report.RSTWriter()
    writer.h1('Anscombe Statistics')
    writer.add(tabulate(table, tablefmt='html', floatfmt='.3f'))
    
    return writer.rst

In [ ]:
def plot(df):
    sns.set(style="ticks")
    g = sns.lmplot(x="x", y="y", col="dataset", hue="dataset", data=df,
               col_wrap=2, ci=None, palette="muted", size=4,
               scatter_kws={"s": 50, "alpha": 1})

    plotting.embellish(g.fig.axes)

In [ ]:
df, table = aggregate()
from IPython.display import display_markdown
display_markdown(generate(table), raw=True)
%matplotlib inline
plot(df)